# Assignment 2: Binary and multinominal outcomes

## Load and Inspect Data

In [1]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np

# The Fishing dataset is in the 'mlogit' R package
dataset_link = "https://vincentarelbundock.github.io/Rdatasets/csv/Ecdat/Fishing.csv"
raw_df = pd.read_csv(dataset_link)

raw_df

,rownames,mode,price,catch,pbeach,ppier,pboat,pcharter,cbeach,cpier,cboat,ccharter,income
0,1,charter,182.930,0.5391,157.930,157.930,157.930,182.930,0.0678,0.0503,0.2601,0.5391,7083.33170
1,2,charter,34.534,0.4671,15.114,15.114,10.534,34.534,0.1049,0.0451,0.1574,0.4671,1249.99980
2,3,boat,24.334,0.2413,161.874,161.874,24.334,59.334,0.5333,0.4522,0.2413,1.0266,3749.99990
3,4,pier,15.134,0.0789,15.134,15.134,55.930,84.930,0.0678,0.0789,0.1643,0.5391,2083.33320
4,5,boat,41.514,0.1082,106.930,106.930,41.514,71.014,0.0678,0.0503,0.1082,0.3240,4583.33200
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177,1178,pier,150.236,0.4522,150.236,150.236,250.746,285.746,0.5333,0.4522,0.2413,1.0266,3749.99990
1178,1179,beach,235.436,0.5333,235.436,235.436,392.946,427.946,0.5333,0.4522,0.2413,1.0266,6250.00130
1179,1180,pier,65.036,0.4522,65.036,65.036,108.546,143.546,0.5333,0.4522,0.2413,1.0266,1249.99980
1180,1181,beach,36.636,0.5333,36.636,36.636,61.146,96.146,0.5333,0.4522,0.1665,0.3975,416.66668


(a) Generate a dummy variable taking 1 if the individual chose to fish on a charter, and
0 otherwise. Run a logit regression using the charter dummy as dependent variable
and prices of the beach, boat, charter, and the catch rate of beach, pier, boat and
charter and also income as independent variables.

In [2]:
X, y = raw_df.assign(
    charter_dummy = (raw_df['mode'] == 'charter').astype(int)
).pipe(
    lambda df: (
        df[['pbeach', 'pboat', 'pcharter', 'cbeach', 'cpier', 'cboat', 'ccharter', 'income']],
        df['charter_dummy']
    )
)
print("Value Counts for y:\n{}".format(y.value_counts()))

print("Descriptive Statistics for X:")
X.describe()


Value Counts for y:
charter_dummy
0    730
1    452
Name: count, dtype: int64
Descriptive Statistics for X:


,pbeach,pboat,pcharter,cbeach,cpier,cboat,ccharter,income
count,1182.000000,1182.000000,1182.000000,1182.000000,1182.000000,1182.000000,1182.000000,1182.000000
mean,103.422005,55.256570,84.379244,0.241011,0.162224,0.171215,0.629368,4099.337054
std,103.641042,62.713444,63.544650,0.190752,0.160390,0.209789,0.706114,2461.964060
min,1.290000,2.290000,27.290000,0.067800,0.001400,0.000200,0.002100,416.666680
25%,26.656500,13.122000,42.896000,0.067800,0.050300,0.023300,0.021900,2083.333200
50%,74.628000,33.534000,61.607000,0.253700,0.078900,0.089700,0.421600,3749.999900
75%,144.144000,72.402000,102.774000,0.533300,0.149800,0.241300,1.026600,5416.666700
max,843.186000,666.110000,691.110000,0.533300,0.452200,0.736900,2.310100,12499.998000


In [3]:
result = sm.Logit(y, X).fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.627697
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          charter_dummy   No. Observations:                 1182
Model:                          Logit   Df Residuals:                     1174
Method:                           MLE   Df Model:                            7
Date:                Fri, 23 May 2025   Pseudo R-squ.:                 0.05642
Time:                        17:27:06   Log-Likelihood:                -741.94
converged:                       True   LL-Null:                       -786.30
Covariance Type:            nonrobust   LLR p-value:                 2.261e-16
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
pbeach         0.0053      0.001      6.040      0.000       0.004       0.007
pboat         -0.0089      0.006     -1.561      0.119      -0.020       0.002
pcharter       0.0048      0.005      0.883      0.377      -0.006       0.016
cbeach        -0.1844      0.600     -0.307      0.758      -1.360       0.991
cpier         -1.3335      0.674     -1.980      0.048      -2.654      -0.013
cboat         -2.3521      0.879     -2.675      0.007      -4.076      -0.629
ccharter       0.8531      0.265      3.220      0.001       0.334       1.372
income        -0.0002   3.69e-05     -5.460      0.000      -0.000      -0.000
==============================================================================
"""

(b) Compute MEM, AME, and OR and interpret your results.

In [4]:
# Marginal Effects at the Mean (MEM)
mem = result.get_margeff(at='mean').summary()
print("\nMarginal Effects at the Mean (MEM):\n", mem)



Marginal Effects at the Mean (MEM):
         Logit Marginal Effects       
Dep. Variable:          charter_dummy
Method:                          dydx
At:                              mean
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
pbeach         0.0012      0.000      6.105      0.000       0.001       0.002
pboat         -0.0021      0.001     -1.553      0.121      -0.005       0.001
pcharter       0.0011      0.001      0.880      0.379      -0.001       0.004
cbeach        -0.0434      0.141     -0.308      0.758      -0.320       0.233
cpier         -0.3141      0.159     -1.977      0.048      -0.625      -0.003
cboat         -0.5540      0.207     -2.681      0.007      -0.959      -0.149
ccharter       0.2009      0.062      3.229      0.001       0.079       0.323
income     -4.749e-05   8.61e-06     -5.517      0.000   -6.44e-05   -3.06e-05


In [5]:
# Average Marginal Effects (AME)
ame = result.get_margeff(at='overall').summary()
print("\nAverage Marginal Effects (AME):\n", ame)



Average Marginal Effects (AME):
         Logit Marginal Effects       
Dep. Variable:          charter_dummy
Method:                          dydx
At:                           overall
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
pbeach         0.0012      0.000      6.400      0.000       0.001       0.002
pboat         -0.0020      0.001     -1.565      0.118      -0.004       0.000
pcharter       0.0011      0.001      0.884      0.377      -0.001       0.003
cbeach        -0.0405      0.132     -0.307      0.758      -0.299       0.218
cpier         -0.2928      0.147     -1.993      0.046      -0.581      -0.005
cboat         -0.5165      0.191     -2.708      0.007      -0.890      -0.143
ccharter       0.1873      0.057      3.277      0.001       0.075       0.299
income     -4.427e-05   7.75e-06     -5.716      0.000   -5.95e-05   -2.91e-05


Marginal effects overall (i.e. averaged) and at mean levels are mostly similar for all predictors.

All predictors except `pboat`, `pcharter` and `cbeach` are statistically significant at the $5 \%$ level.
This is interesting as it implies that fishers do not care about the price of the charter when deciding to charter or not to charter a boat for fishing.

Instead they only seem to care about the price of fishing at the beach, which increases the likelihood of chartering.
Still, the actual change in likelihood is rather small.

In [6]:
# Odds Ratios
or_ = np.exp(result.params)
print("\nOdds Ratios (OR):\n", or_)




Odds Ratios (OR):
 pbeach      1.005288
pboat       0.991153
pcharter    1.004847
cbeach      0.831617
cpier       0.263544
cboat       0.095165
ccharter    2.346937
income      0.999798
dtype: float64


For obvious reasons, the catch rate on charters increases the likelihood of chartering the most.
A $1$-point increase in the `ccharter` more than doubles the $P(Charter)$.

On the other hand, a one point increase in `cboat` would divide the $P(Charter)$ by $10$.

Note that all of these assume all else being equal, although there may be a strong correlation among the predictors which we didn't check.


### 2. 


In [7]:
import pyreadr

result = pyreadr.read_r('assignments/Assignment 2/holiday.rds')

# done! let's see what we got
print(result.keys()) # let's check what objects we got
df1 = result[None] # extract the pandas data frame for object df1
df1

odict_keys([None])


,holiday,salary,price.lake,price.coast,price.hiking,price.mtb,recov.lake,recov.coast,recov.hiking,recov.mtb
rownames,,,,,,,,,,
1,mtb,5388.887800,150.0335,134.2405,181.6195,365.860,0.064410,0.037725,0.273105,1.0782
2,mtb,1499.999867,14.3583,12.8469,12.1141,69.068,0.099655,0.033825,0.165270,0.9342
3,hiking,3166.666600,153.7803,137.5929,27.9841,118.668,0.506635,0.339150,0.253365,2.0532
4,coast,2055.555467,14.3773,12.8639,64.3195,169.860,0.064410,0.059175,0.172515,1.0782
5,hiking,3722.221333,101.5835,90.8905,47.7411,142.028,0.064410,0.037725,0.113610,0.6480
...,...,...,...,...,...,...,...,...,...,...
1178,coast,3166.666600,142.7242,127.7006,288.3579,571.492,0.506635,0.339150,0.253365,2.0532
1179,lake,4833.334200,223.6642,200.1206,451.8879,855.892,0.506635,0.339150,0.253365,2.0532
1180,coast,1499.999867,61.7842,55.2806,124.8279,287.092,0.506635,0.339150,0.253365,2.0532
